In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 70, 16
    model = Sequential()
    model.add(Conv1D(5, 10, strides = 1,  input_shape = (6250,19)))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size = 2, strides = 2))

    model.add(Conv1D(10, 10, strides = 1))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size = 2, strides = 2))

    model.add(Conv1D(10, 10, strides = 1))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size = 2, strides = 2))

    model.add(Conv1D(15, 5, strides = 1))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size = 2, strides = 2))

    model.add(Flatten())

    model.add(Dense(20))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))

    model.add(Dense(10))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))

    model.compile(loss= 'binary_crossentropy' , optimizer= Adam(lr = 0.0001), metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/70
58/58 [==============================] - 1s 10ms/step - loss: 0.7498 - accuracy: 0.5137
Epoch 2/70
58/58 [==============================] - 1s 10ms/step - loss: 0.7251 - accuracy: 0.5104
Epoch 3/70
58/58 [==============================] - 1s 10ms/step - loss: 0.7169 - accuracy: 0.5148
Epoch 4/70
58/58 [==============================] - 1s 10ms/step - loss: 0.7098 - accuracy: 0.5115
Epoch 5/70
58/58 [==============================] - 1s 9ms/step - loss: 0.7099 - accuracy: 0.5192
Epoch 6/70
58/58 [==============================] - 1s 9ms/step - loss: 0.7040 - accuracy: 0.5170
Epoch 7/70
58/58 [==============================] - 1s 10ms/step - loss: 0.7048 - accuracy: 0.4786
Epoch 8/70
58/58 [==============================] - 1s 10ms/step - loss: 0.7032 - accuracy: 0.5115
Epoch 9/70
58/58 [==============================] - 1s 9ms/step - loss: 0.7008 - accuracy: 0.5060
Epoch 10/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6995 - accuracy: 0.5137
Epoch 11/70


58/58 [==============================] - 1s 10ms/step - loss: 0.6960 - accuracy: 0.5027
Epoch 14/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6963 - accuracy: 0.4984
Epoch 15/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6981 - accuracy: 0.4918
Epoch 16/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6955 - accuracy: 0.4830
Epoch 17/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6956 - accuracy: 0.5049
Epoch 18/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6953 - accuracy: 0.4874
Epoch 19/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6947 - accuracy: 0.4852
Epoch 20/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6951 - accuracy: 0.4940
Epoch 21/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6953 - accuracy: 0.4918
Epoch 22/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6947 - accuracy: 0.5049
Epoch 23/70


58/58 [==============================] - 1s 10ms/step - loss: 0.6940 - accuracy: 0.5148
Epoch 26/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6938 - accuracy: 0.5038
Epoch 27/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6937 - accuracy: 0.4962
Epoch 28/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6939 - accuracy: 0.4863
Epoch 29/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6938 - accuracy: 0.5049
Epoch 30/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6936 - accuracy: 0.5170
Epoch 31/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6938 - accuracy: 0.4786
Epoch 32/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6937 - accuracy: 0.5082
Epoch 33/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6936 - accuracy: 0.4896
Epoch 34/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6935 - accuracy: 0.5422
Epoch 35/70
58/

58/58 [==============================] - 1s 9ms/step - loss: 0.6941 - accuracy: 0.5005
Epoch 38/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6941 - accuracy: 0.5279
Epoch 39/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6942 - accuracy: 0.4929
Epoch 40/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6939 - accuracy: 0.5016
Epoch 41/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6939 - accuracy: 0.5016
Epoch 42/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6941 - accuracy: 0.4940
Epoch 43/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6937 - accuracy: 0.5060
Epoch 44/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6937 - accuracy: 0.4786
Epoch 45/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6941 - accuracy: 0.5115
Epoch 46/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6937 - accuracy: 0.4951
Epoch 47/70
58/5

58/58 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.4830
Epoch 50/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6933 - accuracy: 0.4885
Epoch 51/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6933 - accuracy: 0.5071
Epoch 52/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6933 - accuracy: 0.4819
Epoch 53/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.4666
Epoch 54/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.4907
Epoch 55/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.4874
Epoch 56/70
58/58 [==============================] - 1s 9ms/step - loss: 0.6933 - accuracy: 0.5016
Epoch 57/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.5093
Epoch 58/70
58/58 [==============================] - 1s 10ms/step - loss: 0.6933 - accuracy: 0.4984
Epoch 59/70
58/5